# Lab | Data Cleaning and Formatting

In this lab, we will be working with the customer data from an insurance company, which can be found in the CSV file located at the following link: https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv


# Challenge 1: Data Cleaning and Formatting

## Exercise 1: Cleaning Column Names

To ensure consistency and ease of use, standardize the column names of the dataframe. Start by taking a first look at the dataframe and identifying any column names that need to be modified. Use appropriate naming conventions and make sure that column names are descriptive and informative.

*Hint*:
- *Column names should be in lower case*
- *White spaces in column names should be replaced by `_`*
- *`st` could be replaced for `state`*

In [2]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')

new_columns = [column.lower().replace(' ','_') for column in df.columns]

df.columns = new_columns

df.rename(columns={'st':'state'},inplace=True)

print(df.columns)

Index(['customer', 'state', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'policy_type', 'vehicle_class', 'total_claim_amount'],
      dtype='object')


## Exercise 2: Cleaning invalid Values

The dataset contains columns with inconsistent and incorrect values that could affect the accuracy of our analysis. Therefore, we need to clean these columns to ensure that they only contain valid data.

Note that this exercise will focus only on cleaning inconsistent values and will not involve handling null values (NaN or None).

*Hint*:
- *Gender column contains various inconsistent values such as "F", "M", "Femal", "Male", "female", which need to be standardized, for example, to "M" and "F".*
- *State abbreviations be can replaced with its full name, for example "AZ": "Arizona", "Cali": "California", "WA": "Washington"*
- *In education, "Bachelors" could be replaced by "Bachelor"*
- *In Customer Lifetime Value, delete the `%` character*
- *In vehicle class, "Sports Car", "Luxury SUV" and "Luxury Car" could be replaced by "Luxury"*

In [3]:
df['gender']=df['gender'].str.lower()

df['gender'] = df['gender'].replace({
    'm': 'M',
    'f': 'F',
    'femal': 'F',
    'male': 'M',
    'female': 'F'
})

df['gender'].value_counts()

df['state'] = df['state'].replace({
    'AZ': 'Arizona',
    'Cali':'California',
    'WA':'Washington'
})

df['state'].value_counts()

df['education'] = df['education'].replace({
    'Bachelors':'Bachelor'
})

df['education'].value_counts()

# Primero, convertimos todo a string para evitar problemas (ya que hay valores NaN)
df['customer_lifetime_value'] = df['customer_lifetime_value'].astype(str)

# Luego, eliminanos el símbolo % y convertimos a número, ignorando los 'nan' que se crearon
df['customer_lifetime_value'] = df['customer_lifetime_value'].str.replace('%', '', regex=False)

df['customer_lifetime_value'].head(10)

df['vehicle_class'] = df['vehicle_class'].replace({
    'Sports Car': 'Luxury',
    'Luxury SUV':'Luxury',
    'Luxury Car':'Luxury'
})

df['vehicle_class'].value_counts()


vehicle_class
Four-Door Car    576
Two-Door Car     205
SUV              199
Luxury            91
Name: count, dtype: int64

## Exercise 3: Formatting data types

The data types of many columns in the dataset appear to be incorrect. This could impact the accuracy of our analysis. To ensure accurate analysis, we need to correct the data types of these columns. Please update the data types of the columns as appropriate.

It is important to note that this exercise does not involve handling null values (NaN or None).

*Hint*:
- *Customer lifetime value should be numeric*
- *Number of open complaints has an incorrect format. Look at the different values it takes with `unique()` and take the middle value. As an example, 1/5/00 should be 5. Number of open complaints is a string - remember you can use `split()` to deal with it and take the number you need. Finally, since it should be numeric, cast the column to be in its proper type.*

In [4]:
# Convertimos las cadenas a float, convirtiendo los 'nan' en NaN reales
df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')
df['customer_lifetime_value'].dtype

def extraer_mayor(valor):
    if pd.isna(valor):
        return None
    numeros = list(map(int, valor.split('/')))
    return max(numeros)

# Aplicamos extraer_mayor a nuestra columna
df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(extraer_mayor)
df['number_of_open_complaints'].value_counts()

number_of_open_complaints
1.0    968
2.0     50
3.0     34
4.0     13
5.0      6
Name: count, dtype: int64

## Exercise 4: Dealing with Null values

Identify any columns with null or missing values. Identify how many null values each column has. You can use the `isnull()` function in pandas to find columns with null values.

Decide on a strategy for handling the null values. There are several options, including:

- Drop the rows or columns with null values
- Fill the null values with a specific value (such as the column mean or median for numerical variables, and mode for categorical variables)
- Fill the null values with the previous or next value in the column
- Fill the null values based on a more complex algorithm or model (note: we haven't covered this yet)

Implement your chosen strategy to handle the null values. You can use the `fillna()` function in pandas to fill null values or `dropna()` function to drop null values.

Verify that your strategy has successfully handled the null values. You can use the `isnull()` function again to check if there are still null values in the dataset.

Remember to document your process and explain your reasoning for choosing a particular strategy for handling null values.

After formatting data types, as a last step, convert all the numeric variables to integers.

In [5]:
# Vemos los valores nulos que tienen las columnas
df.isnull().sum()

# Podemos rellenar los valores con la media u otro valor
# df.fillna(df.mean(numeric_only=True))

# Quitamos todas las filas con valores nulos
df_without_na = df.dropna().copy()

#Comprobamos ahora que las columnas no tienen valores nulos
df_without_na.isnull().sum()

# Seleccionamos las columnas numéricas
columnas_numericas = df_without_na.select_dtypes(include=['float', 'int']).columns

# Convertimos las columnas numéricas a int
df_without_na[columnas_numericas] = df_without_na[columnas_numericas].astype(int)

#Comprobamos los tipos de datos del dataframe
df_without_na.dtypes

customer                     object
state                        object
gender                       object
education                    object
customer_lifetime_value       int64
income                        int64
monthly_premium_auto          int64
number_of_open_complaints     int64
policy_type                  object
vehicle_class                object
total_claim_amount            int64
dtype: object

## Exercise 5: Dealing with duplicates

Use the `.duplicated()` method to identify any duplicate rows in the dataframe.

Decide on a strategy for handling the duplicates. Options include:
- Dropping all duplicate rows
- Keeping only the first occurrence of each duplicated row
- Keeping only the last occurrence of each duplicated row
- Dropping duplicates based on a subset of columns
- Dropping duplicates based on a specific column

Implement your chosen strategy using the `drop_duplicates()` function.

Verify that your strategy has successfully handled the duplicates by checking for duplicates again using `.duplicated()`.

Remember to document your process and explain your reasoning for choosing a particular strategy for handling duplicates.

Save the cleaned dataset to a new CSV file.

*Hint*: *after dropping duplicates, reset the index to ensure consistency*.

In [ ]:
# Vemos las filas de duplicados
duplicados = df_without_na[df_without_na.duplicated()]
print(duplicados)

# Optamos por borrar las filas con duplicados
df_sin_duplicados = df_without_na.drop_duplicates()
df_sin_duplicados.duplicated()
df_sin_duplicados.head()

# Grabamos el nuevo DataFrame en un nuevo fichero
df_sin_duplicados.to_csv('file2.csv',index=False)

# Leemos el nuevo fichero y comprobamos
df_nuevo = pd.read_csv('file2.csv')

df_nuevo.head()

Empty DataFrame
Columns: [customer, state, gender, education, customer_lifetime_value, income, monthly_premium_auto, number_of_open_complaints, policy_type, vehicle_class, total_claim_amount]
Index: []


,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,QZ44356,Arizona,F,Bachelor,697953,0,94,1,Personal Auto,Four-Door Car,1131
1,AI49188,Nevada,F,Bachelor,1288743,48767,108,1,Personal Auto,Two-Door Car,566
2,WW63253,California,M,Bachelor,764586,0,106,1,Corporate Auto,SUV,529
3,GA49547,Washington,M,High School or Below,536307,36357,68,1,Personal Auto,Four-Door Car,17
4,OC83172,Oregon,F,Bachelor,825629,62902,69,1,Personal Auto,Two-Door Car,159


# Bonus: Challenge 2: creating functions on a separate `py` file

Put all the data cleaning and formatting steps into functions, and create a main function that performs all the cleaning and formatting.

Write these functions in separate .py file(s). By putting these steps into functions, we can make the code more modular and easier to maintain.

*Hint: autoreload module is a utility module in Python that allows you to automatically reload modules in the current session when changes are made to the source code. This can be useful in situations where you are actively developing code and want to see the effects of changes you make without having to constantly restart the Python interpreter or Jupyter Notebook kernel.*

In [1]:
# def cleaning_columns_names(df):
#     """
#     Limpia y estandariza los nombres de las columnas de un DataFrame.

#     Operaciones realizadas:
#     ------------------------
#     1. Convierte todos los nombres de las columnas a minúsculas.
#     2. Reemplaza los espacios en los nombres de columnas por guiones bajos ('_').
#     3. Renombra específicamente la columna 'st' para que pase a llamarse 'state'.
#     4. Actualiza el DataFrame con los nuevos nombres de columnas.

#     Parámetros:
#     ------------
#     df : pandas.DataFrame
#         El DataFrame cuyos nombres de columnas serán limpiados y estandarizados.

#     Retorna:
#     --------
#     pandas.DataFrame
#         El mismo DataFrame de entrada pero con los nombres de las columnas modificados.

#     """
#     # Ponemos los nombres de columnas en minúsculas y reemplazamos espacios por guiones
#     new_columns = [column.lower().replace(' ','_') for column in df.columns]

#     df.columns = new_columns

#     # Renombrarmos la columna 'st' como 'state'
#     df.rename(columns={'st':'state'},inplace=True)

#     return df

# def cleaning_invalid_values(df):
#     """
#     Limpia y normaliza valores inconsistentes en varias columnas de un DataFrame.

#     Operaciones realizadas:
#     ------------------------
#     1. Convierte todos los valores de la columna 'gender' a minúsculas.
#     2. Normaliza los valores de 'gender' para que sólo existan 'M' (masculino) y 'F' (femenino).
#     3. Reemplaza abreviaturas de estados ('AZ', 'Cali', 'WA') por sus nombres completos ('Arizona', 'California', 'Washington').
#     4. Corrige el valor 'Bachelors' en la columna 'education' para uniformizarlo como 'Bachelor'.
#     5. Elimina el símbolo '%' en los valores de la columna 'customer_lifetime_value' y los deja en formato numérico como texto (sin convertir todavía a float).
#     6. Agrupa las categorías de vehículos de alta gama ('Sports Car', 'Luxury SUV', 'Luxury Car') en una única categoría: 'Luxury'.

#     Parámetros:
#     ------------
#     df : pandas.DataFrame
#         El DataFrame que contiene los datos a limpiar y normalizar.

#     Retorna:
#     --------
#     pandas.DataFrame
#         El mismo DataFrame, pero con los valores corregidos y estandarizados.

#     """
#     # Pasamos a minusculas todos los valores de la columna 'gender'
#     df['gender']=df['gender'].str.lower()

#     # Normalizamos los valores para tener sólo 'M' y 'F'
#     df['gender'] = df['gender'].replace({
#         'm': 'M',
#         'f': 'F',
#         'femal': 'F',
#         'male': 'M',
#         'female': 'F'
#     })

#     # Reemplazmos los valores de estados como AZ, Cali y WA por sus valores completos
#     df['state'] = df['state'].replace({
#         'AZ': 'Arizona',
#         'Cali':'California',
#         'WA':'Washington'
#     })

#     # En la columna de 'Education', reemplazamos 'Bachelors' por 'Bacherlor'
#     df['education'] = df['education'].replace({
#         'Bachelors':'Bachelor'
#     })

#     # Transformamos los valores de la columna 'customer_lifetime_value' para eliminar el carácter '%'
#     # Primero, convertimos todo a string para evitar problemas (ya que hay valores NaN)
#     df['customer_lifetime_value'] = df['customer_lifetime_value'].astype(str)

#     # Luego, eliminanos el símbolo % y convertimos a número, ignorando los 'nan' que se crearon
#     df['customer_lifetime_value'] = df['customer_lifetime_value'].str.replace('%', '', regex=False)

#     # Trasnsformamos los valores de la columna 'vehicle_class' para agrupar a todos los coches de alta gama como 'Luxury'
#     df['vehicle_class'] = df['vehicle_class'].replace({
#         'Sports Car': 'Luxury',
#         'Luxury SUV':'Luxury',
#         'Luxury Car':'Luxury'
#     })

#     return df

# def extraer_mayor(valor):
#     """
#     Extrae el mayor número de una cadena de números separados por '/'.

#     Parámetros:
#     -----------
#     valor : str o None
#         Cadena de texto con números enteros separados por barras ("/"), 
#         o un valor nulo (NaN).

#     Retorna:
#     --------
#     int o None
#         El número entero más grande encontrado en la cadena.
#         Si el valor es NaN, devuelve None.

#     Ejemplos:
#     ---------
#     >>> extraer_mayor("1/3/5")
#     5

#     >>> extraer_mayor("2/2/2")
#     2

#     >>> extraer_mayor(None)
#     None
#     """
#     if pd.isna(valor):
#         return None
#     numeros = list(map(int, valor.split('/')))
#     return max(numeros)

# def formatting_data_types(df):
#     """
#     Formatea y corrige tipos de datos específicos en un DataFrame.

#     Operaciones realizadas:
#     ------------------------
#     1. Convierte los valores de la columna 'customer_lifetime_value' de texto (que pueden incluir errores o valores no numéricos) a valores numéricos tipo float.
#        - Los valores que no se puedan convertir se transformarán en NaN (Not a Number).

#     2. Aplica la función `extraer_mayor` sobre la columna 'number_of_open_complaints', 
#        que transforma cadenas de números separados por '/' en el número mayor de la secuencia.
    
#     3. Devuelve el DataFrame modificado.

#     Parámetros:
#     ------------
#     df : pandas.DataFrame
#         El DataFrame que contiene las columnas 'customer_lifetime_value' y 'number_of_open_complaints' a formatear.

#     Retorna:
#     --------
#     pandas.DataFrame
#         El mismo DataFrame de entrada pero con los tipos de datos corregidos en las columnas mencionadas.

#     """
#     # Convertimos las cadenas de la columna 'customer_lifetime_value' a float, convirtiendo los 'nan' en NaN reales
#     df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')
   
#     # Aplicamos extraer_mayor a la columna 'number_of_open_complaints'
#     df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(extraer_mayor)
   
#     return df 

# def dealing_with_null_values(df):
#     """
#     Elimina los valores nulos del DataFrame y convierte las columnas numéricas a tipo entero.

#     Pasos:
#     1. Elimina todas las filas que contienen valores nulos en cualquier columna.
#     2. Realiza una copia explícita del DataFrame resultante para evitar el warning SettingWithCopyWarning.
#     3. Identifica todas las columnas de tipo numérico (float o int).
#     4. Convierte las columnas numéricas a tipo entero (int), eliminando así posibles decimales residuales.

#     Parámetros:
#     ----------
#     df : pandas.DataFrame
#         El DataFrame original que puede contener valores nulos y columnas numéricas flotantes.

#     Retorna:
#     -------
#     pandas.DataFrame
#         Un nuevo DataFrame sin valores nulos y con las columnas numéricas convertidas a enteros.
#     """

#     # Quitamos todas las filas con valores nulos
#     df = df.dropna().copy()

#     # Seleccionamos las columnas numéricas
#     columnas_numericas = df.select_dtypes(include=['float', 'int']).columns

#     # Convertimos las columnas numéricas a int
#     df[columnas_numericas] = df[columnas_numericas].astype(int)

#     return df

# def dealing_with_duplicates(df):
#     """
#     Elimina las filas duplicadas de un DataFrame.

#     Pasos:
#     1. Identifica las filas duplicadas en el DataFrame.
#     2. Elimina las filas duplicadas, conservando únicamente la primera aparición de cada registro único.
#     3. Devuelve un nuevo DataFrame limpio, sin duplicados.

#     Parámetros:
#     ----------
#     df : pandas.DataFrame
#         El DataFrame original que puede contener filas duplicadas.

#     Retorna:
#     -------
#     pandas.DataFrame
#         Un nuevo DataFrame en el que se han eliminado todas las filas duplicadas.
#     """
    
#     # Optamos por borrar las filas con duplicados conservando la primera aparición
#     df_sin_duplicados = df.drop_duplicates()

#     return df_sin_duplicados

# def save_dataframe(df,file_url):
#     """
#     Guarda un DataFrame en un archivo CSV.

#     Pasos:
#     1. Convierte el DataFrame en un archivo CSV.
#     2. Guarda el archivo en la ubicación especificada, sin incluir los índices de las filas.

#     Parámetros:
#     ----------
#     df : pandas.DataFrame
#         El DataFrame que se desea guardar en un archivo.
    
#     file_url : str
#         Ruta y nombre del archivo donde se guardará el DataFrame en formato CSV.

#     Retorna:
#     -------
#     None
#         Esta función no retorna ningún valor. Solo guarda el archivo en la ubicación indicada.
#     """
    
#     # Grabamos el nuevo DataFrame en un nuevo fichero
#     df.to_csv(file_url,index=False)



import pandas as pd
from functions_cleanning import *

df = (
    pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')
    .pipe(cleaning_columns_names)
    .pipe(cleaning_invalid_values)
    .pipe(formatting_data_types)
    .pipe(dealing_with_null_values)
    .pipe(dealing_with_duplicates)
)

save_dataframe(df, 'file3.csv')


# Bonus: Challenge 3: Analyzing Clean and Formated Data

You have been tasked with analyzing the data to identify potential areas for improving customer retention and profitability. Your goal is to identify customers with a high policy claim amount and a low customer lifetime value.

In the Pandas Lab, we only looked at high policy claim amounts because we couldn't look into low customer lifetime values. If we had tried to work with that column, we wouldn't have been able to because customer lifetime value wasn't clean and in its proper format. So after cleaning and formatting the data, let's get some more interesting insights!

Instructions:

- Review the statistics again for total claim amount and customer lifetime value to gain an understanding of the data.
- To identify potential areas for improving customer retention and profitability, we want to focus on customers with a high policy claim amount and a low customer lifetime value. Consider customers with a high policy claim amount to be those in the top 25% of the total claim amount, and clients with a low customer lifetime value to be those in the bottom 25% of the customer lifetime value. Create a pandas DataFrame object that contains information about customers with a policy claim amount greater than the 75th percentile and a customer lifetime value in the bottom 25th percentile.
- Use DataFrame methods to calculate summary statistics about the high policy claim amount and low customer lifetime value data. To do so, select both columns of the dataframe simultaneously and pass it to the `.describe()` method. This will give you descriptive statistics, such as mean, median, standard deviation, minimum and maximum values for both columns at the same time, allowing you to compare and analyze their characteristics.

In [11]:
# Revisamos las estadísticas de las columnas 'total_claim_amount' y 'customer_lifetime_value'
df[['total_claim_amount', 'customer_lifetime_value']].describe()

# Averiguamos los clientes con alta cantidad de reclamaciones (están en el 25% superior)
percentil_75 = df['total_claim_amount'].quantile(0.75)

clientes_top_reclamacion = df[df['total_claim_amount']>percentil_75]

# Averiguamos los clientes con menor valor de tiempo de vida
percentil_25 = df['customer_lifetime_value'].quantile(0.25)

clientes_menor_valor = df[df['customer_lifetime_value']<percentil_25]

# Por tanto los peores clientes serán los de mayor cantidad de reclamaciones y menor valor en tiempo de vida
peores_clientes = df[(df['total_claim_amount']>percentil_75) & (df['customer_lifetime_value']<percentil_25) ]

# Calculamos las estadisticas de esos clientes en las columnas 'total_claim_amount' y 'customer_lifetime_value'
peores_clientes[['total_claim_amount', 'customer_lifetime_value']].describe()



,total_claim_amount,customer_lifetime_value
count,35.000000,35.000000
mean,711.571429,343009.085714
std,145.349333,57540.930008
min,537.000000,228759.000000
25%,631.500000,299913.000000
50%,679.000000,366737.000000
75%,757.000000,386920.500000
max,1185.000000,407913.000000
